### This is a toy notebook that will always work (and give confidence to the user)

In [17]:
from ortools.sat.python import cp_model
import pandas as pd
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utility import JobPartialSolutionPrinter


### Create the JobPartialSolutionPrinter which helps to print all the solutions. The last will be optimal

### Generate the toy dataset. We have some jobs which are important and some which are not. We simulate the dataset of availability and also the dataset of skills

In [18]:
NUM_WEEKS = 13
all_members = ['Jeff', 'John', 'Jill', 'Jack', 'whee', 'mary']
all_jobs = ['Guitar', 'Piano', 'Leader', 'Waterboy']
crucial_jobs = ['Guitar', 'Piano', 'Leader']
non_crucial_jobs = ['Waterboy']
all_weeks = [str(i) for i in range(NUM_WEEKS)]
input_col_names = ['name'] + all_weeks

# Simulate availability constraints (some members are unavailable on some weeks)

simulated_availability = [[True if random.random() > 0.05 else False for _ in all_weeks] for _ in all_members]
availability_df = pd.DataFrame(simulated_availability, columns=(all_weeks), index=all_members)

simulated_skills = [[True if random.random() > 0.5 else False for _ in all_jobs] for _ in all_members]
skills_df = pd.DataFrame(simulated_skills, columns=(all_jobs), index=all_members)

display(availability_df)
display(skills_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12
Jeff,True,True,True,True,True,True,True,True,True,True,True,True,False
John,True,False,True,True,True,True,True,True,True,True,True,True,True
Jill,True,True,True,True,True,True,True,True,True,True,True,True,True
Jack,True,True,True,True,True,True,True,True,True,True,True,True,True
whee,True,True,True,True,True,True,True,True,True,True,True,True,True
mary,True,True,True,True,True,True,True,True,True,True,True,True,True


,Guitar,Piano,Leader,Waterboy
Jeff,False,True,True,False
John,True,True,True,True
Jill,True,True,True,False
Jack,True,False,True,False
whee,False,False,True,False
mary,False,True,False,False


### The model has the following constraints (updated 02/03/2025)

- Each crucial job is assigned to only 1 member
- Each non-crucial job is assigned to at most one member per week.
- Each member does at most one job per week.
- Availability constraints
- Skill constraints
- Add constraint that members cannot be rostered thrice in a row

### The model optimises the following (updated 02/03/2025)
- Maximise the total assignments across the board
- Minimise variance in assignments per person

In [19]:
# Creates the model
model = cp_model.CpModel()

# Creates job assignment variables.
shifts = {}
for m in all_members:
    for w in all_weeks:
        for j in all_jobs:
            shifts[(m, w, j)] = model.NewBoolVar(f"shift_m{m}_w{w}_j{j}")

# Each job is assigned to exactly one member per week.
for w in all_weeks:
    for j in crucial_jobs:
        model.AddExactlyOne(shifts[(m, w, j)] for m in all_members)

# Each non crucial job is assigned to at most one member per week.
for w in all_weeks:
    for j in non_crucial_jobs:
        model.AddAtMostOne(shifts[(m, w, j)] for m in all_members)

# Each member does at most one job per week.
for m in all_members:
    for w in all_weeks:
        model.AddAtMostOne(shifts[(m, w, j)] for j in all_jobs)

# Add availability constraints
for m in all_members:
    for w in all_weeks:
        if not availability_df.loc[m, str(w)]: 
            for j in all_jobs:
                model.Add(shifts[(m, w, j)] == 0)

# Add skill constraints
for m in all_members:
    for j in all_jobs:
        if not skills_df.loc[m, j]: 
            for w in all_weeks:
                model.Add(shifts[(m, w, j)] == 0)
                
# Add constraint that members cannot be rostered thrice
for m in all_members:
    for w in all_weeks[:-2]:  # Ensure we don't go out of bounds
        for j in all_jobs:
            model.Add(
                shifts[(m, w, j)] + shifts[(m, str(int(w) + 1), j)] + shifts[(m, str(int(w) + 2), j)] <= 2
            )


# Fairness constraint: Minimize variance in assignments
total_assignments = {}
for m in all_members:
    total_assignments[m] = model.NewIntVar(0, len(all_weeks) * len(all_jobs), f"total_assignments_{m}")
    model.Add(total_assignments[m] == sum(shifts[(m, w, j)] for w in all_weeks for j in all_jobs))

# Compute the mean assignments (rounded)
avg_assignments = len(all_weeks) * len(all_jobs) // len(all_members)

# Deviation from mean
deviation = {}
for m in all_members:
    deviation[m] = model.NewIntVar(0, len(all_weeks) * len(all_jobs), f"deviation_{m}")
    model.Add(deviation[m] >= total_assignments[m] - avg_assignments)
    model.Add(deviation[m] >= avg_assignments - total_assignments[m])

squared_deviation = {}
for m in all_members:
    squared_deviation[m] = model.NewIntVar(0, (len(all_weeks) * len(all_jobs)) ** 2, f"squared_deviation_{m}")
    model.AddMultiplicationEquality(squared_deviation[m], [deviation[m], deviation[m]])

# Minimize sum of squared deviations to reduce variance
model.Minimize(-sum(total_assignments[m] for m in all_members) * 10 + sum(squared_deviation[m] for m in all_members))
                


# Creates the solver and solve.
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solver.parameters.enumerate_all_solutions = False

solution_printer = JobPartialSolutionPrinter(
    shifts, all_members, all_weeks, all_jobs, total_assignments, verbose = 1
)

status = solver.Solve(model, solution_printer)

# Check if the optimal solution was found
if status == cp_model.OPTIMAL:
    print("\nOptimal solution found!")
    # Print the solution table
    solution_df = pd.DataFrame()
    solution_df["Job"] = all_jobs
    for w in all_weeks:
        week_list = []
        for j in all_jobs:
            job_filled = False
            for m in all_members:
                if solver.Value(shifts[(m, w, j)]):
                    week_list.append(m)
                    job_filled = True
            if not job_filled:
                week_list.append(np.nan)
        solution_df[f"Week {w}"] = week_list
    display(solution_df)
    solution_df.to_csv('data/final_solution.csv')
    # Print total assignments per member
    print("\nTotal Assignments per Member:")
    for m in all_members:
        total = solver.Value(total_assignments[m])  # Retrieve total assignments
        print(f"  - {m}: {total} jobs assigned")


elif status == cp_model.FEASIBLE:
    print("\nFeasible solution found, but not necessarily optimal.")
else:
    print("\nNo solution found.")




Solution 1


,Job,Week 0,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11,Week 12
0,Guitar,John,Jack,John,John,Jill,John,John,Jill,John,John,Jill,John,John
1,Piano,mary,Jill,mary,mary,John,mary,mary,John,mary,mary,John,mary,mary
2,Leader,whee,Jeff,whee,whee,Jeff,whee,whee,Jeff,whee,whee,Jeff,whee,whee
3,Waterboy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Total Assignments per Member:
  - Jeff: 4 jobs assigned
  - John: 12 jobs assigned
  - Jill: 4 jobs assigned
  - Jack: 1 jobs assigned
  - whee: 9 jobs assigned
  - mary: 9 jobs assigned

Solution 2


,Job,Week 0,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11,Week 12
0,Guitar,John,Jill,John,Jack,John,Jill,Jill,John,John,Jack,John,John,Jack
1,Piano,Jill,Jeff,Jill,Jeff,Jeff,John,mary,Jeff,Jeff,Jill,Jeff,Jill,Jill
2,Leader,Jeff,whee,Jeff,whee,Jill,whee,John,Jill,Jill,Jeff,Jack,Jack,John
3,Waterboy,NaN,NaN,NaN,John,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Total Assignments per Member:
  - Jeff: 9 jobs assigned
  - John: 11 jobs assigned
  - Jill: 11 jobs assigned
  - Jack: 5 jobs assigned
  - whee: 3 jobs assigned
  - mary: 1 jobs assigned

Solution 3


...


Total Assignments per Member:
  - Jeff: 7 jobs assigned
  - John: 9 jobs assigned
  - Jill: 8 jobs assigned
  - Jack: 8 jobs assigned
  - whee: 8 jobs assigned
  - mary: 7 jobs assigned

Solution 10


,Job,Week 0,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11,Week 12
0,Guitar,Jill,Jack,Jill,Jack,Jill,Jack,Jill,Jack,Jack,Jill,Jack,Jack,Jill
1,Piano,mary,mary,Jeff,mary,mary,Jill,mary,Jeff,Jill,mary,Jeff,mary,mary
2,Leader,whee,Jeff,whee,Jeff,whee,Jeff,whee,whee,Jeff,whee,whee,Jeff,Jack
3,Waterboy,John,NaN,John,John,NaN,John,John,NaN,John,John,NaN,John,John



Total Assignments per Member:
  - Jeff: 8 jobs assigned
  - John: 9 jobs assigned
  - Jill: 8 jobs assigned
  - Jack: 8 jobs assigned
  - whee: 7 jobs assigned
  - mary: 8 jobs assigned

Optimal solution found!


,Job,Week 0,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11,Week 12
0,Guitar,Jill,Jack,Jill,Jack,Jill,Jack,Jill,Jack,Jack,Jill,Jack,Jack,Jill
1,Piano,mary,mary,Jeff,mary,mary,Jill,mary,Jeff,Jill,mary,Jeff,mary,mary
2,Leader,whee,Jeff,whee,Jeff,whee,Jeff,whee,whee,Jeff,whee,whee,Jeff,Jack
3,Waterboy,John,NaN,John,John,NaN,John,John,NaN,John,John,NaN,John,John



Total Assignments per Member:
  - Jeff: 8 jobs assigned
  - John: 9 jobs assigned
  - Jill: 8 jobs assigned
  - Jack: 8 jobs assigned
  - whee: 7 jobs assigned
  - mary: 8 jobs assigned
